## NK Model

### Introduction

This notebook (...).

### Parameters and settings


All parameter settings and modeling choices go in this section.

In [1]:
# MODEL PARAMETERS
N = 4 # length of each genome
K_i = 2 # ruggedness parameter of the NK model for individual fitness
K_g = K_i # ruggedness parameter of the NK model for groups
G = 50 # number of groups
I = 100 # maximum number of individuals per group
gen_time = 10 # generation time of groups relative tot that of individuals 

# NEUTRALIY
neutrality_i = "NKp" # fitness model at individual level; choose "NK", "NKp", or "NKq"
neutrality_g = "NKp" # fitness model at individual level; choose "NK", "NKp", or "NKq"
p_i = 0.5 # p of NKp for individual-level fitness
p_g = p_i # p of NKp for group-level fitness
q_i = 4 # q of NKq for individual-level fitness
q_g = q_i # q of NKq for group-level fitness

# NETWORK PROPERTIES
"""
choose "r" for sampling with replacement, 
"nr" for without replacement, 
"block" for blockwise interactions
"""
network_i = "r"
network_g = "r"

# SIMULATION PARAMETERS
my_seed = 10 # random seed
t_end = 1000 # end time, in units of individual generation times.

### Packages

In [3]:
#import necessary packages
import numpy as np
import random as rd
import matplotlib.pyplot as plot
from mpl_toolkits.mplot3d import Axes3D
from sklearn.linear_model import LinearRegression

#set style for all plots
plot.style.use("seaborn-v0_8-colorblind")

### Shorthands

Calculate shorthands that will be used throughout the simulation.

In [4]:
B_i = 2**(K_i + 1) # number of hypercube corners for the fitness contributions of each gene
# (= columns in fitness matrix) at the individual level 
B_g = 2**(K_g + 1) # number of hypercube corners for the fitness contributions of each gene
# (= columns in fitness matrix) at the group level

### Global variables

Define and create global variables that will be used thoughout the simulation.

In [23]:
# fitness matrix
fm_i = np.zeros((N, B_i)) # invullen: lege array N x B_j
fm_g = np.zeros((N, B_g)) # invullen: lege arry N x B_g

# epistasis matrices
val = list(range(0, N))

# (initial) genomes/coordinates individual and group level
ind_genomes = np.random.randint(0, 2, size = (I, N)) #randomly generates binary genomes for all individuals 
##ind_genomes_gr = #individuals grouped, not sure yet what smartest way is
##gr_genomes = np.mean(ind_genomes_gr, axis=0,) #takes avg of each column to generate group genome - exact method depends on grouping method

# fitness values
#f_i = # absolute fitness individual level
f_i_comp = np.random.rand(I, N) #randomly generates fitness contributions associated with gene values of all individuals
f_i = np.mean(f_i_comp, axis=1,) #absolute fitness of all individuals (=avg by row of f_i_comp)
f_i = f_i.reshape(-1, 1) 
#f_g = # absolute fitness group level

#w_i = # relative fitness individual level

#w_g = # relative fitness group level

### Function definitions

#### ... for constructing epistasis matrix without repetition

In [ ]:
#functions to create epistasis matrices without repeats
def generate_all_perm_tree(level, nums): 
    """
    recursively generates tree structure with all possible permutations of given 'nums',
    where each number is allowed to move to next position in permutation, excluding repeats
    - level: Current level in the permutation.
    - nums (list): list of numbers to permute.
    returns:
    - dict: nested dictionary representing the permutation tree structure.
        Each key is a number, and corresponding value is the subtree for the next level.
    """
    if len(nums) == 1:
        if level == nums[0]:
            return None
        else:
            return {nums[0]: {}}
    allowed_number = list(nums)
    if level in allowed_number:
        allowed_number.remove(level)
    result = {}
    for number in allowed_number:
        sublevel_number = list(nums)
        if number in sublevel_number:
            sublevel_number.remove(number)
        subtree = generate_all_perm_tree(level + 1, sublevel_number)
        if subtree is not None:
            result[number] = subtree
    if len(result) == 0:
        return None
    return result

def pick_all_moved_perm(all_moved_perm_tree, picked=None):#picks permutation of numbers from previously generated tree, with each number selected only once
    """
    Picks permutation of numbers from previously generated tree, with each number selected only once
    - all_moved_perm_tree (dictionary): The permutation tree generated by generate_all_perm_tree
    - picked: set of numbers already picked.
    Returns:
    - list: representing a permutation of numbers
    """
    if picked is None:
        picked = set()
    allowed_num_set = set(all_moved_perm_tree.keys()) - picked
    if not allowed_num_set:
        return []
    number = rd.choice(list(allowed_num_set))
    picked.add(number)
    l = [number]
    sub_tree = all_moved_perm_tree[number]
    if len(sub_tree) > 0:
        l.extend(pick_all_moved_perm(sub_tree, picked))
    return l

def generate_unique_r(tree, num_rows): 
    """
    Generates an array of unique pairs of numbers, with no number repeated in a row
    - tree (dict):the permutation tree generated by generate_all_moved_perm_tree.
    - num_rows: the number of rows to generate.
    Returns:
    - 2d array representing unique pairs of numbers in each row
    """
    result = []
    for _ in range(num_rows):
        row = list(zip(pick_all_moved_perm(tree), pick_all_moved_perm(tree)))
        while any(x[0] == x[1] for x in row):
            row = list(zip(pick_all_moved_perm(tree), pick_all_moved_perm(tree)))
        result.extend(row)
    return np.array(result[:num_rows])

tree = generate_all_perm_tree(1, range(1, N+1))

#### ... for choosing preferred epistasis

In [27]:
def create_epistasis_matrix_i(): #epistasis matrix individual level
    em = []
    match network_i:
        case ["r"]:
            for row in range(N):
                val = list(range(0, N))
                gene_pair = rd.sample(val[:row] + val[row + 1:], K_i)  
                em_ri.append([row] + gene_pair)
            em_ri = np.array(em_ri)  #first with own gene referenced
            em_ri = em_ri[:, 1:] #without own gene referenced #remove this row or above
        case ["nr"]:
            em = generate_unique_r(tree, N)
        case ["block"]:
            em = #working on it :)
    return em

def create_epistasis_matrix_g(): #epistasis matrix group level
    em = []
    match network_g:
        case ["r"]:
            for row in range(N):
                val = list(range(0, N))
                gene_pair = rd.sample(val[:row] + val[row + 1:], K_g)  #Select K unique genes 
                em_rg.append([row] + gene_pair)  # Row number added to the beginning
            em_rg = np.array(em_rg)  #first with own gene referenced
            em_rg = em_rg[:, 1:] 
        case ["nr"]:
            em = generate_unique_r(tree, N)
        case ["block"]:
            em = #working on it :)
    return em

SyntaxError: invalid syntax (3810782164.py, line 14)

#### ... for constructing fitness landscapes

In [6]:
def create_fitness_matrix_i():
    fm = np.random.rand(N, B_i)
    match neutrality_i:
        case ["NK"]:
            #
            pass
        case ["NKp"]:
            fm = np.where(np.random.rand(*fm.shape) < p_i, 0, fm) 
        case ["NKq"]:
            fm = np.digitize(fm, bins=np.linspace(0, 1, q_i+1), right=True) - 1
    return fm

def create_fitness_matrix_g():
    fm = np.random.rand(N, B_g)
    match neutrality_g:
        case ["NK"]:
            #
            pass
        case ["NKp"]:
            fm = np.where(np.random.rand(*fm.shape) < p_g, 0, fm) 
        case ["NKq"]:
            fm = np.digitize(fm, bins=np.linspace(0, 1, q_g+1), right=True) - 1
    return fm

#### ... for calculating coefficients (ai0 to aij)

In [26]:
def calc_a(K, fm): 
    """
    calculate and return coefficients based on fitness matrix 
    - K: number of rows in the matrix
    - fm: fitness matrix
    Returns:
    - List containing 'a' coefficients for each row of the input matrix
    """
    a_coefficients = []
    for r in fm:
        a = [0.0] * B_g  # initialises list with zeros as floats for each row & X cols
        a[0] = r[0] #set ai0 to Fi0 
        for j in range(1, B_g): 
            sum = 0.0 
            # Calculate next coefs based only on previously calculated coefs
            for l in range(0, j): 
                #check if l equal to bitwise AND of l and j 
                #(ex: 001&101->001 TRUE; 001&100->000 FALSE)
                if l == (l & j): 
                    sum += a[l] 
            a[j] = r[j] - sum #update currect coef
        a_coefficients.append(a) # append new a's into result array
    return a_coefficients

#### ... for calculating fitness values

In [7]:
#fitness functions
def gene_fitness(coefficients, epistasis, genome, gene):
    """
    Calculate the fitness component of a specific gene in a genome
    - coefficients (array): coefficient matrix as calculated by calc_a
    - epistasis ("): Epistasis matrix representing interactions between genes
    - genome ("): genome values
    - gene (int): index of specific gene for which fitness component is calculated
    Returns:
    - fitness component of the specified gene in the genome (float)
    """
    result = 0

    for j in range(coefficients.shape[1]):
        contribution = coefficients[gene, j] * genome[gene] ** (1 & j)
        for k in range(epistasis.shape[1]):
            epi_index = epistasis[gene, k]
            epi_value = genome[epi_index]
            product_term = epi_value ** ((2**k & j) / 2**k)
            contribution *= product_term

        result += contribution
    return result

#function to calculate all fitness components within genome
def genome_fitness(coefficients, epistasis, genome):
    """
    Calculate fitness components for all genes within a genome 
    return:
    array containing fitness components for each gene in the genome
    """
    fit_vals = np.zeros(len(genome))

    for gene in range(len(genome)):
        fit_vals[gene] = gene_fitness(coefficients, epistasis, genome, gene)
    return fit_vals

#function to calculate fitness components for all genes in all genomes
def calculate_fitness(coefficients, epistasis, genomes):
    """
    Calculate the fitness components for all genes in all genomes
    Return:
    2D array containing fitness components for each gene (cols) in each genome (rows)
    """
    M, N = len(genomes), len(genomes[0])
    fit_val = np.zeros((M, N))

    for group in range(M):
        fit_val[group, :] = genome_fitness(coefficients, epistasis, genomes[group])
    return fit_val

#### ... for running simulations

In [ ]:
def initialize_genomes():
    #sth

def update_rates(): 
    #sth

def choose_event(rates): 
    #sth

def execute_reaction(event):
    #sth

### Tests

In [ ]:
#to-do ma/di
#testing for multilinearity continuous NK model: vary one coordinate/gene value of a group genome, while keeping others constant:
ind = 1 #pick specific group based on id 
varying_gene_vals = np.linspace(0, 1, 100) #100 regularly spaced values between 0 and 1
constant_gene_vals = coords[ind-1, 1:] #takes coordinates/gene values from the chosen genome
var_coords = np.zeros((100, N)) #creates array with 100 rows for storing coordinates
var_coords[:, 0] = varying_gene_vals 
var_coords[:, 1:] = np.tile(constant_gene_vals, (100, 1)) #result is array of shape (100, N), with new values in first column and constant coords in other columns

a__rep = np.tile(coef[ind-1], (100, 1)) #Create new array for a_shape with values selected genome repeated 100 times
em_rep = np.tile(em[ind-1], (100, 1)) #wrong

#fitness funciton that iterates through all rows of the new array with 100 rows with varied coord
def calculate_fitness_var(coefficients, epistasis, genomes):
    fit_val = np.zeros((100, N))

    for group in range(100):
        genome = genomes[group]
        for gene in range(N):
            result = 0  # Initialize with 0
            for j in range(coefficients.shape[1]):  # summation & multiplication, bitwise 
                contribution = coefficients[gene, j] * genome[gene] ** (1 & j)
                for k in range(epistasis.shape[1]):
                    epi_index = epistasis[gene, k]
                    epi_value = genome[epi_index]
                    product_term = epi_value ** ((2 ** k & j) / 2 ** k)
                    contribution *= product_term

                result += contribution

            fit_val[group, gene] = result
    return fit_val

fit_var = calculate_fitness_var(a_shape_rep, em2_inc, var_coords) #fitness components
avg_fit = np.mean(fit_var, axis=1) #average fitness
print(fit_var)

## The simulation

In [19]:
rd.seed(my_seed) # set random seed

# CONSTRUCT FITNESS LANDSCAPES 
fm_i = create_fitness_matrix_i()
fm_g = create_fitness_matrix_g()
# (and so on)


# INIITIALIZE
genomes = initialize_genomes()
t = 0
# RUN SIMULATIONS
while t < t_end :
    rates = update_rates()
    propensity = sum(rates)
    tau = np.random.exponential(scale=1 / propensity) # waiting time for next reaction
    t = t + tau
    event = choose_event(rates)
    execute_reaction(event)


NameError: name 'initialize_genomes' is not defined

## Results